# PaniniFS Research - Analyse Dhātu Accélérée\n\nAnalyse linguistique haute performance avec GPU/TPU\n\n## 🚀 Configuration GPU/TPU

In [ ]:
# Configuration initiale\n!nvidia-smi\n!pip install torch torchvision torchaudio transformers\n!pip install pandas numpy matplotlib seaborn\n!pip install nltk spacy scikit-learn\n\nimport torch\nimport numpy as np\nimport pandas as pd\nfrom transformers import pipeline\n\n# Vérification GPU\ndevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\nprint(f'Device: {device}')\nif torch.cuda.is_available():\n    print(f'GPU: {torch.cuda.get_device_name()}')

## 📂 Connexion au Workspace PaniniFS

In [ ]:
# Connexion au repository GitHub\n!git clone https://github.com/stephanedenis/PaniniFS-Research.git\n%cd PaniniFS-Research\n\n# Import des modules PaniniFS\nimport sys\nsys.path.append('./src')\n\nfrom analysis.analyseur_molecules_semantiques import MoleculeAnalyzer\nfrom dhatu.aspect_dhatu import AspectAnalyzer\nfrom corpus.universal_atoms_extractor import AtomExtractor

## 🧠 Analyse Dhātu avec Transformers

In [ ]:
class ColabDhatuAnalyzer:\n    def __init__(self, device='cuda'):\n        self.device = device\n        self.sentiment_pipeline = pipeline('sentiment-analysis', device=0 if device=='cuda' else -1)\n        self.ner_pipeline = pipeline('ner', device=0 if device=='cuda' else -1)\n        \n    def analyze_semantic_patterns(self, texts: list):\n        \"\"\"Analyse sémantique accélérée par GPU\"\"\"\n        results = []\n        \n        for text in texts:\n            # Analyse de sentiment\n            sentiment = self.sentiment_pipeline(text)\n            \n            # Reconnaissance d'entités\n            entities = self.ner_pipeline(text)\n            \n            # Extraction de patterns dhātu\n            dhatu_patterns = self.extract_dhatu_patterns(text)\n            \n            results.append({\n                'text': text,\n                'sentiment': sentiment,\n                'entities': entities,\n                'dhatu_patterns': dhatu_patterns\n            })\n            \n        return results\n        \n    def extract_dhatu_patterns(self, text):\n        \"\"\"Extraction de patterns dhātu spécifiques\"\"\"\n        # Patterns aspectuels\n        aspectual_markers = ['was', 'were', 'will', 'would', 'has', 'have', 'had']\n        modal_markers = ['can', 'could', 'may', 'might', 'should', 'must']\n        \n        patterns = {\n            'aspectual': [marker for marker in aspectual_markers if marker in text.lower()],\n            'modal': [marker for marker in modal_markers if marker in text.lower()]\n        }\n        \n        return patterns\n\n# Initialisation\nanalyzer = ColabDhatuAnalyzer(device=str(device))

## 📊 Traitement Batch de Corpus

In [ ]:
# Chargement de corpus depuis PaniniFS\nimport json\n\ndef load_corpus_data():\n    corpus_files = []\n    data_dir = Path('./data')\n    \n    if data_dir.exists():\n        corpus_files = list(data_dir.glob('corpus*.json'))\n        \n    print(f'Corpus trouvés: {len(corpus_files)}')\n    return corpus_files\n\ndef process_corpus_batch(corpus_files, batch_size=32):\n    \"\"\"Traitement par batch pour optimiser GPU\"\"\"\n    all_results = []\n    \n    for corpus_file in corpus_files:\n        print(f'Traitement: {corpus_file.name}')\n        \n        with open(corpus_file, 'r', encoding='utf-8') as f:\n            data = json.load(f)\n            \n        # Extraction des textes\n        texts = []\n        if isinstance(data, dict):\n            texts = [str(v) for v in data.values() if isinstance(v, str)]\n        elif isinstance(data, list):\n            texts = [str(item) for item in data if isinstance(item, str)]\n            \n        # Traitement par batch\n        for i in range(0, len(texts), batch_size):\n            batch = texts[i:i+batch_size]\n            batch_results = analyzer.analyze_semantic_patterns(batch)\n            all_results.extend(batch_results)\n            \n            print(f'  Batch {i//batch_size + 1}: {len(batch)} textes traités')\n            \n    return all_results\n\n# Exécution\ncorpus_files = load_corpus_data()\nif corpus_files:\n    results = process_corpus_batch(corpus_files)\n    print(f'\\n✅ Analyse terminée: {len(results)} éléments traités')

## 📈 Visualisation des Résultats

In [ ]:
import matplotlib.pyplot as plt\nimport seaborn as sns\nfrom collections import Counter\n\ndef visualize_dhatu_analysis(results):\n    \"\"\"Visualise les résultats d'analyse dhātu\"\"\"\n    \n    # Analyse des sentiments\n    sentiments = [r['sentiment'][0]['label'] for r in results if r['sentiment']]\n    sentiment_counts = Counter(sentiments)\n    \n    # Patterns dhātu\n    aspectual_patterns = []\n    modal_patterns = []\n    \n    for r in results:\n        aspectual_patterns.extend(r['dhatu_patterns']['aspectual'])\n        modal_patterns.extend(r['dhatu_patterns']['modal'])\n        \n    aspectual_counts = Counter(aspectual_patterns)\n    modal_counts = Counter(modal_patterns)\n    \n    # Visualisations\n    fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n    \n    # Sentiments\n    axes[0, 0].pie(sentiment_counts.values(), labels=sentiment_counts.keys(), autopct='%1.1f%%')\n    axes[0, 0].set_title('Distribution des Sentiments')\n    \n    # Patterns aspectuels\n    if aspectual_counts:\n        axes[0, 1].bar(aspectual_counts.keys(), aspectual_counts.values())\n        axes[0, 1].set_title('Marqueurs Aspectuels')\n        axes[0, 1].tick_params(axis='x', rotation=45)\n    \n    # Patterns modaux\n    if modal_counts:\n        axes[1, 0].bar(modal_counts.keys(), modal_counts.values())\n        axes[1, 0].set_title('Marqueurs Modaux')\n        axes[1, 0].tick_params(axis='x', rotation=45)\n    \n    # Entités\n    entity_types = []\n    for r in results:\n        for entity in r['entities']:\n            entity_types.append(entity['entity'])\n    \n    if entity_types:\n        entity_counts = Counter(entity_types)\n        top_entities = dict(entity_counts.most_common(10))\n        axes[1, 1].bar(top_entities.keys(), top_entities.values())\n        axes[1, 1].set_title('Top 10 Types d\'Entités')\n        axes[1, 1].tick_params(axis='x', rotation=45)\n    \n    plt.tight_layout()\n    plt.show()\n    \n    # Statistiques\n    print('\\n📊 STATISTIQUES DHĀTU:')\n    print(f'Total textes analysés: {len(results)}')\n    print(f'Patterns aspectuels uniques: {len(aspectual_counts)}')\n    print(f'Patterns modaux uniques: {len(modal_counts)}')\n    print(f'Types d\'entités: {len(Counter(entity_types))}')\n\n# Visualisation\nif 'results' in locals() and results:\n    visualize_dhatu_analysis(results)

## 💾 Export des Résultats

In [ ]:
# Export vers Google Drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\n# Sauvegarde\nimport datetime\n\ntimestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')\nexport_file = f'/content/drive/MyDrive/panini_dhatu_analysis_{timestamp}.json'\n\nif 'results' in locals():\n    with open(export_file, 'w', encoding='utf-8') as f:\n        json.dump(results, f, indent=2, ensure_ascii=False)\n    \n    print(f'✅ Résultats exportés: {export_file}')\n    print(f'📊 {len(results)} analyses sauvegardées')\n\n# Création d'un rapport\nreport = {\n    'timestamp': timestamp,\n    'device': str(device),\n    'total_analyses': len(results) if 'results' in locals() else 0,\n    'gpu_info': torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU',\n    'corpus_files_processed': len(corpus_files) if 'corpus_files' in locals() else 0\n}\n\nreport_file = f'/content/drive/MyDrive/panini_report_{timestamp}.json'\nwith open(report_file, 'w', encoding='utf-8') as f:\n    json.dump(report, f, indent=2)\n\nprint(f'📋 Rapport sauvegardé: {report_file}')